In [ ]:
# from tqdm.notebook import tqdm
# from tqdm import tqdm

import os
os.environ['USE_PYGEOS'] = '0'

# Data management
import pandas as pd
import geopandas as gpd
import numpy as np


# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.features import Choropleth
from folium.plugins import MarkerCluster

# Preprocessing
from sklearn.preprocessing import StandardScaler
import umap


# I/O
import gc
import io, requests
import zipfile, shutil
import joblib

# tqdm().pandas()

# data_path = 'C:/Users/demo/Desktop/Lattitude/datas/'
data_path = 'datas'
os.makedirs(data_path, exist_ok=True)

# Datas

In [ ]:
# Load the data for French garge points
file_name = 'dataset_charge_points.feather'

bornes = gpd.read_feather(os.path.join(data_path, file_name))

# Load the data for the communes
file_name = 'dataset_communes.feather'

datas = gpd.read_feather(os.path.join(data_path, file_name))

-------------------------

# Create some metric on electrics stuff

In [ ]:
datas['VE_pct'] = datas.nb_vp_rechargeables_el / datas.nb_vp
datas['VE_per_inhab'] = datas.nb_vp_rechargeables_el / datas.PMUN
datas['VE_per_ha'] = datas.nb_vp_rechargeables_el / datas.surf_ha


# Select a region

In [ ]:
display(set(datas.region_name), set(datas.columns) )

In [ ]:
REGION = 'Grand Est'

In [ ]:
datas = datas.query("region_name == @REGION")
bornes = bornes.query("region_name == @REGION")

In [ ]:
datas.sort_values(by='date_arrete',ascending=True, inplace=True)

---------------------------------------------------------------------------------------

# Create HTML content for map popup

In [ ]:
# list of columns from datas_ to be displayed
datas_info_cols = datas.columns.tolist()
datas_info_cols.remove('geometry')
datas_info_cols.remove('wikipedia')



texts = []

# loop on rows
for index, row in datas.iterrows():
    text = ''
    for col in datas_info_cols:
        text += f'<b>{col}:</b> {row[col]}<br>'
    texts.append(text)

datas['html_popup'] = texts   
del texts

datas.html_popup.head(2)

In [ ]:
# list of columns from datas_ to be displayed
bornes_info_cols = bornes.columns.tolist()
bornes_info_cols.remove('geometry')

# Column names generator 
def split_list(list_a, chunk_size):
  for i in range(0, len(list_a), chunk_size):
    yield list_a[i:i + chunk_size]


# Mise en page
num_cols = 3
width = 20
px = np.ceil(width / num_cols / 2)

texts = []

# Loop on rows
for index, row in bornes.iterrows():
    text = f'<table style="width:{width}%"><tr>'
    for n in range(num_cols):
       text += f'<td style="font-weight:bold">{n}</td>'
    
    for cols in split_list(bornes_info_cols,num_cols):
            text += '<tr>'
            for col in cols:
                    text += f'<th style="width:{px}%"><b>{col}:</b><br> {row[col]}</th>' 
            text  += '</tr>' 
    text += '</table>'
    texts.append(text)

bornes['html_popup'] = texts   
del texts

bornes.html_popup.head(2)

------------------------------------------------------

# Map

In [ ]:
datas.crs

In [ ]:
from branca.colormap import linear, LinearColormap

# Define the color map
colors = [ 'red', 'green', 'blue']
bins = np.array([0, 1, 5, 10, 25, 50, 100]) / 100

labels = [1,2,3,4,5,6]

cmap = LinearColormap(colors=colors, vmin=1, vmax=6)

'''color argument of Icon should be one of: 
{'red', 'darkred', 'gray', 'blue', 'black', 'darkpurple', 'white', 'darkblue', 
 'purple', 'lightred', 'green', 'orange', 'cadetblue', 'beige', 'lightblue', 
 'lightgray', 'darkgreen', 'pink', 'lightgreen'}.'''
icon_labels = ['darkred','lightred','lightgreen','darkgreen','lightblue','blue']

# Function to make a column color 
def make_color(df, col='VE_per_inhab', color_type=None):
    if color_type:
        color =pd.cut( df[col], #.apply(lambda x:  x ** (1/3)),
                        bins=bins, 
                        labels=icon_labels)
    else:
        color =pd.cut( df[col], #.apply(lambda x:  x ** (1/3)),
                            bins=bins, 
                            labels=labels).apply(cmap)
    return color





In [ ]:
def make_color_by_date(df, date_col='date_arrete', col='VE_per_inhab', color_type=None, icons=None):
    if icons:
        df['color'] = 'white'
    else:
        df['color'] = '#000000'

    for date in df[date_col].unique():
        df.loc[df[date_col] == date,'color'] = make_color(df.loc[df[date_col] == date], col, icons)
    
    return df


display(make_color_by_date(datas, 'VE_per_inhab').color.value_counts())

display(make_color_by_date(datas, 'VE_per_inhab', icons='ok').color.value_counts())    


In [ ]:
'''
icons from https://fontawesome.com/v4/icons/

'''
from folium.plugins import MarkerCluster
import math


def make_map(com_df, pdc_df, color_col='VE_per_inhab'):


    # Create a folium map centered on the first row of datas_
    datas_centroid = com_df.iloc[0].geometry.centroid
    # datas_centroid

    # create a folium map
    m = folium.Map(location=[datas_centroid.y,datas_centroid.x], zoom_start=6, crs='EPSG3857')

    # Create a marker cluster layer for the data
    cluster_com = MarkerCluster(name='Communes', )
    cluster_color = MarkerCluster(name='colors')

    # Create colors by binning the VE_per_inhab column
    com_df = make_color_by_date(com_df, 'VE_per_inhab',icons='XXX')    

    # communes markers
    for index, row in com_df.iterrows():

        # communes child markers
        popup = folium.Popup(row.html_popup, parse_html=False)
        cluster_com.add_child(folium.Marker(location=[
                                                    row.geometry.centroid.y,
                                                    row.geometry.centroid.x
                                                ], 
                                            popup=popup, 
                                            tooltip=row.date_arrete,   #.strftime('%d/%m/%Y'),
                                            icon=folium.Icon(
                                                        prefix='fa', 
                                                        icon='institution', 
                                                        color=row.color
                                                        )
                                            )
        )
        
        
    cluster_com.add_to(m)
    cluster_color.add_to(m)

    # Create a marker cluster layer for the data
    cluster_bdr = MarkerCluster(name='Points de charge')


    # bornes markers
    for index, row in pdc_df.iterrows():
        popup = folium.Popup(row.html_popup, parse_html=False)
        cluster_bdr.add_child(folium.Marker(location=[row.geometry.y, row.geometry.x], 
                                            popup=popup, 
                                            tooltip='pdc infos',
                                            icon=folium.Icon(prefix='fa', icon='bolt', color='blue')))
        
    cluster_bdr.add_to(m)

    # Add a layer control to the map
    folium.LayerControl().add_to(m)

    return m

m = make_map(datas, bornes)

display(m)



In [ ]:
maps_path = 'datas/maps'
os.makedirs(maps_path, exist_ok=True)

file_name = f'Carte region {REGION}.html'

m.save(os.path.join(maps_path,file_name))

del m

-----------------------

In [ ]:
# import minify_html

# def schrink_html(file_name):
#     input_file = os.path.join(maps_path, file_name + '.html')
#     raw = open(input_file, "r").read()
#     input_size = len(raw)

#     schrinked = minify_html.minify(raw,
#         # do_not_minify_doctype= True,
#         ensure_spec_compliant_unquoted_attribute_values= True,
#         keep_closing_tags= True,
#         keep_html_and_head_opening_tags= True,
#         keep_spaces_between_attributes= True,
#         keep_comments= False,
#         minify_css= True,
#         minify_js= True,
#         remove_bangs= True,
#         remove_processing_instructions= True,
#         )
#     output_size = len(schrinked)

#     print(f'Size  {input_file}: {input_size} | output: {output_size} | {output_size / input_size}')
#     with open(os.path.join(maps_path, file_name + '_min.html'), "w") as output_file:
#         output_file.write(schrinked)


# schrink_html(os.path.join(maps_path,file_name))